In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
import pywt

In [2]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [28]:
def reconstruct_detail(coeffs, w_name, level):
    """
    Reconstruct detail coefficients at a specific level.

    Parameters:
    -----------
    coeffs : list
        Wavelet coefficients from wavedec.
    w_name : str
        Wavelet name.
    level : int
        The level to reconstruct.

    Returns:
    --------
    array_like
        Reconstructed detail coefficients.
    """
    coeffs_d = [np.zeros_like(c) for c in coeffs]
    coeffs_d[level] = coeffs[level]
    return pywt.waverec(coeffs_d, w_name)

In [62]:
coeffs = pywt.wavedec(ts1, 'db3', level=2)
det = pywt.upcoef('d', coeffs=coeffs[1], wavelet='db3', level=2, take=len(ts1)) # details reconstruction at level of decomp. 
x = np.sort(np.abs(det))
print(int(np.floor(len(x) * 0.98)))
#v2p100 = x[int(np.floor(len(x) * 0.98))]

980


In [56]:
v2p100

0.032815588962967074

In [3]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_glscf(y, alpha, beta, tau = 'tau'):
    """
    """
    # Set tau to first zero-crossing of the autocorrelation function with the input 'tau'
    if tau == 'tau':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Take magnitudes of time-delayed versions of the time series
    y1 = np.abs(y[:-tau])
    y2 = np.abs(y[tau:])


    p1 = np.mean(np.multiply((y1 ** alpha), (y2 ** beta)))
    p2 = np.multiply(np.mean(y1 ** alpha), np.mean(y2 ** beta))
    p3 = np.sqrt(np.mean(y1 ** (2*alpha)) - (np.mean(y1 ** alpha))**2)
    p4 = np.sqrt(np.mean(y2 ** (2*beta)) - (np.mean(y2 ** beta))**2)

    glscf = (p1 - p2) / (p3 * p4)

    return glscf    


In [11]:
CO_glscf(ts1, 0.9, 0.4, 3)

0.25284356429315435

In [13]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_glscf import CO_glscf

def CO_fzcglscf(y, alpha, beta, maxtau = None):
    """
    """
    N = len(y) # the length of the time series

    if maxtau is None:
        maxtau = N
    
    glscfs = np.zeros(maxtau)

    for i in range(1, maxtau+1):
        tau = i

        glscfs[i-1] = CO_glscf(y, alpha, beta, tau)
        if (i > 1) and (glscfs[i-1]*glscfs[i-2] < 0):
            # Draw a straight line between these two and look at where it hits zero
            out = i - 1 + glscfs[i-1]/(glscfs[i-1]-glscfs[i-2])
            return out
    
    return maxtau # if the function hasn't exited yet, set output to maxtau 


In [17]:
CO_fzcglscf(ts3, 0.1, 0.2, 100)

2.727416384317647

In [84]:
def DN_pleft(y, th = 0.1):
    """
    Distance from the mean at which a given proportion of data are more distant.
    """
    p = np.quantile(np.abs(y - np.mean(y)), 1-th, method='hazen')

    # A proportion, th, of the data lie further than p from the mean
    out = p/np.std(y, ddof=1)

    return out

In [85]:
DN_pleft(ts1)

1.3935047122505273